# Lab 2

In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [ ]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
!ls /users/trush/CSC496/PageRank/Lab2

In [ ]:
auth_log = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data")
auth_log.take(5)

## 1. How many failed attempts to access the server as root are there? List all countries from which these attempts where carried out.

In [ ]:
# Total failed attempts to login
#total_failed_attempts = auth_log.filter(lambda x: "sshd" in x)
#total_failed_attempts.count()

# Number of failed attempts to login as root
failed_attempts = auth_log.filter(lambda x: "root" in x and "sshd" in x)
#failed_attempts.take(5)
failed_attempts.count()

In [ ]:
import IP2Location
import re

database = IP2Location.IP2Location(os.path.join("data", "/users/trush/CSC496/PageRank/Lab2/IP-COUNTRY.BIN"))

def get_country_name(x):
    ip = re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=x)[1]
    rec = database.get_all(x).country_long
    #print(type(rec))
    return rec

#get_country_name('218.92.0.248')

In [ ]:
failed_root_countries = failed_attempts.map(lambda line: get_country_name(line))
#failed_root_countries.take(5)
failed_root_countries.distinct.collect()

## 2. How many failed attempts to access the server as non-root users are there? What are the attempted usernames? List all countries from which these attempts where carried out. 

In [ ]:
# Number of failed attempts to login as non-root
failed_nonroot = auth_log.filter(lambda x: "root" not in x and "sshd" in x and "user unknown" not in x)
#failed_nonroot.take(5)
failed_nonroot.count()

In [ ]:
failed_nonroot_countries = failed_attempts_nonroot.map(lambda line: get_country_name(line))
#failed_nonroot_countries.take(5)
failed_nonroot_countries.distinct().collect()

In [ ]:
#failed_nonroot_usernames = failed_attempts_nonroot.map(lambda line: re.split(pattern=r'(user )', string=line).rsplit(' from'))
#failed_nonroot_usernames.take(5)
#failed_nonroot_usernames.distinct().collect()

## 3. Through out the durations of the log files, which date has the highest number of attack attempts? Anything interesting about that particular date?

In [ ]:
# Something to do with Chinese Labor Day (May 1)
dates = auth_log.map(lambda line: (re.split(pattern = r'\S{3}\W{1,2}\D{2}', string=line)[0], 1))
dates.take(5)
#countsByKey = dates.countByKey()
#countsByKey